In [10]:

import pandas as pd
root = "/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/"


In [11]:
#read in data


#go terms for orthology analysis from: Supplementary material/annotation_orthogroup_genes.xlsx

go_terms = pd.read_csv(root + "internal data/go_terms_formatted.csv")

#omics data sources
omics_sources = pd.read_excel(root + "internal data/external data sources.xlsx")

#floc_cor_genes /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Analysis of Clement-Ziza RNA-seq/filter_corr.csv
floc_cor_genes = pd.read_csv(root + "Bence folder/Analysis of Clement-Ziza RNA-seq/filter_corr.csv")

#structure_sequence_homology /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Structure_and_Sequence_comparisons/final_analysis.csv
structure_sequence_homology = pd.read_csv(root + "Bence folder/Structure_and_Sequence_comparisons/final_analysis.csv")

#srb11 DE genes /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Analysis of Clement-Ziza RNA-seq/DESeq2/DE_results.csv
srb11_DE_genes = pd.read_csv(root + "Bence folder/Analysis of Clement-Ziza RNA-seq/DESeq2/DE_results.csv")

#WI phenotypes /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Analysis of Jeffares phenotypes/growth_vs_adhesion.csv
WI_phenotypes = pd.read_csv(root + "Bence folder/Analysis of Jeffares phenotypes/growth_vs_adhesion.csv")

#segregant genotypes/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/internal data/updated_genotype_matrix_final_pos.tsv
segregant_genotypes = pd.read_csv(root + "internal data/updated_genotype_matrix_final_pos.tsv", sep="\t")


wi_adh_across_media = pd.read_excel(root + "Bence folder/Image processing - Adhesion assay/condition_plot_df.xlsx")


#primers
primers=pd.read_excel(root + "internal data/Primers.xlsx")

#screen_raw from root + "internal data/Deletion screen raw results.csv
screen_raw = pd.read_csv(root + "internal data/Deletion screen raw results.csv")

#screen enrichment from root + Bence folder/Angeli/angeli/bioneer_angeli_results_recent.txt
screen_enrichment = pd.read_csv(root + "Bence folder/Angeli/angeli/bioneer_angeli_results_recent.txt", sep="\t")

#screen_final root + "Bence folder/Bioneer screen/final_results_screen.csv"
screen_final = pd.read_csv(root + "Bence folder/Bioneer screen/final_results_screen.csv")






In [12]:
#Join all of these into a single xlsx file as different sheets. Also give a header to each sheet
# which is "# Title: " + the name of the sheet
# correspondance: bence.kover.19@ucl.ac.uk or j.bahler@ucl.ac.uk

# Import the necessary libraries
import pandas as pd

# Define the file path for the output xlsx file
output_file = root + "Supplementary material/Supplementary table.xlsx"

# Create a Pandas Excel writer using the output file path
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

# Define a dictionary containing the dataframes and their corresponding sheet names
dataframes = {
    '1_go_terms': go_terms,
    '2_omics_sources': omics_sources,
    '3_segregant_genotypes': segregant_genotypes,
    '4_structure_sequence_homology': structure_sequence_homology,
    '5_WI_adhesion_across_media' : wi_adh_across_media,
     '6_floc_cor_genes': floc_cor_genes,
    '7_srb11_DE_genes': srb11_DE_genes,
    '8_Screen_raw_results': screen_raw,
    '9_Screen_enrichment': screen_enrichment,
    '10_Screen_final_results': screen_final
}

# Loop through the dictionary and write each dataframe to a separate sheet in the output file
for sheet_name, df in dataframes.items():
    # Add a header to the sheet name
    sheet_name = sheet_name
    title = "# Title: " + sheet_name
    correspondance = "# Correspondance: bence.kover.19@ucl.ac.uk or j.bahler@ucl.ac.uk"
    
    # Write the dataframe to the sheet
    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=2)
    
    # Get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # Add the title and correspondance to the sheet
    worksheet.write(0, 0, title)
    worksheet.write(1, 0, correspondance)
    
    # Move the column names down
    for i, col in enumerate(df.columns):
        worksheet.write(2, i, col)

# Save the Excel file
writer.save()

# Now assemble .pdf of all images

In [13]:
!pip install pillow fpdf2
!pip install PyPDF2



In [15]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None 
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Image as ReportLabImage, Paragraph, PageBreak
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Image as ReportLabImage
import os
import PyPDF2


def compress_pdf(pdf_path):
    compressed_path = pdf_path.replace(".pdf", "_compressed.pdf")
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_writer = PyPDF2.PdfWriter()

        for page in pdf_reader.pages:
            pdf_writer.add_page(page)

        with open(compressed_path, "wb") as compressed_pdf:
            pdf_writer.write(compressed_pdf)

    os.remove(pdf_path)  # Remove the original uncompressed PDF
    os.rename(compressed_path, pdf_path)  # Rename the compressed PDF to original name



def create_pdf_from_images(image_paths, output_path, n_main_figs, n_supp_figs):
    doc = SimpleDocTemplate(output_path, pagesize=A4)
    story = []

    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path)
        width, height = img.size
        aspect_ratio = height / float(width)
        new_width = 500  # You can adjust this width as needed for A4 layout
        new_height = new_width * aspect_ratio
        if new_height > 600:
            new_height = 600
            new_width = new_height / aspect_ratio
        img.thumbnail((new_width, new_height), Image.ANTIALIAS)

        reportlab_img = ReportLabImage(image_path, width=new_width, height=new_height)
        
        
        # Add a caption to the PDF document
        if i < n_main_figs:
            caption = f"<strong>Figure {i+1}</strong>"
        elif i < n_main_figs + n_supp_figs:
            caption = f"<strong>Supplementary Figure {i-n_main_figs+1}</strong>"
        else:
            caption = ""
        caption_style = ParagraphStyle(name='Caption', fontSize=16, leading=14)
        story.append(Paragraph(caption, caption_style))
        #paragraph
        story.append(Paragraph("<br/>", caption_style))
        story.append(Paragraph("<br/>", caption_style))
        story.append(Paragraph("<br/>", caption_style))
        story.append(reportlab_img)
        story.append(PageBreak())


    doc.build(story)
    compress_pdf(output_path)

    



image_paths = [
    root + "Figures/Figure 0/figure0.png",
    root + "Figures/Figure 1/figure 1.png",
    root + "Figures/Figure 2/fig2.png",
    root + "Figures/Figure 3/figure3.png",
    root + "Figures/Figure 4/figure4.png",
    root + "Figures/Figure 5/final_final plot.png",
    root + "Figures/Supplementary figure 1./supfig1.png",
    root + "Figures/Supplementary figure 2./supp_fig2.png",
    root + "Figures/Supplementary figure 3./supp_fig3.png",
    root + "Figures/Supplementary figure 4./suppfig4.png",
    root + "Figures/Supplementary figure 6./supp_fig6.png",

    root + "Figures/Supplementary figure 7./supp_fig7.png",
    root + "Figures/Supplementary figure 10./supp_fig10.png",
    root + "Figures/Supplementary figure 8./supfig8.png",

    root + "Figures/Supplementary figure 9./supfig9.png",
    root + "Figures/Supplementary figure 9./supfig9.png",

    
    root + "Figures/Supplementary figure 11./supp_fig11.png",
    root + "Figures/Supplementary figure 12./supp_fig12.png",
    root + "Figures/Supplementary figure 13./supfig13.png",

]
output_path = root + "Supplementary material/figures.pdf"

create_pdf_from_images(image_paths, output_path,6,13)





/var/folders/mb/vdj4y7t13zd3tzlk99b8ycd40000gn/T/ipykernel_45304/3822915502.py:42: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail((new_width, new_height), Image.ANTIALIAS)
